In [1]:
import sys
sys.path.append("..")

from citibikeAPI import *
from utils import *
import openrouteservice as ors
import folium

import sys
sys.path.append("..")
import seaborn as sns
import matplotlib.pyplot as plt
from citibikeAPI import *

client = ors.Client(key='5b3ce3597851110001cf62487d33115ea15e483bb81a0b4cfcc899cb')

In [2]:
def findMostCommonRoutesByStation(trip_data):
	station_data = getAllStationData()
	#we only want trips that dont start and end at the same station
	trips = trip_data[trip_data['start station id'] != trip_data['end station id']].value_counts(['start station name','end station name'])
	#find unique starting stations
	unique_start_stations = list(set([trips.index[i][0] for i in range(len(trips))]))
	adjacencyDict = {}
	for station in unique_start_stations: 
		try: 
			id = station_data.index[station_data['name']==station][0]
			top_ten_routes = trips[station][:10]
			edges = []
			for route in top_ten_routes.index: 
				edges.append(station_data.index[station_data['name']==route][0])
			adjacencyDict[id] = edges
		#sometimes we get an index error if a station went online / offline since the last data pull 
		except IndexError: 
			pass 
	return adjacencyDict

In [3]:
d = getAllStationData()
trip_data = pd.read_csv('../../raw/202003-citibike-tripdata.csv')
adjacenyDict = findMostCommonRoutesByStation(trip_data)


In [4]:
d.head()
locData = d
locData = locData.reset_index()
locData.head()

,station_id,name,lat,lon,capacity,legacy_id,num_docks_available,num_bikes_disabled,num_ebikes_available,num_bikes_available,num_docks_disabled,station_status,is_renting,is_returning,last_reported,is_installed
0,c71cca54-17f6-42bc-ba94-3f5bd9c70197,Bridge St & Water St,40.702969,-73.984691,47,4590,0,4,0,43,0,active,1,1,1681249318,1
1,038b7369-9c6e-4eb2-8569-f7461443302f,Fulton St & Williams Ave,40.677090,-73.900800,21,4912,2,2,0,17,0,active,1,1,1681249429,1
2,66dce8db-0aca-11e7-82f6-3863bb44ef7c,Montrose Ave & Bushwick Ave,40.707678,-73.940162,42,3074,24,0,0,17,0,active,1,1,1681249430,1
3,517409d3-a94a-4140-a6de-ff0de9221ed2,Westchester Ave & E 156 St,40.817802,-73.905203,23,3983,18,1,0,3,0,active,1,1,1681249428,1
4,66ddeaff-0aca-11e7-82f6-3863bb44ef7c,West End Ave & W 94 St,40.794165,-73.974124,31,3307,29,0,0,0,0,active,1,1,1681249426,1


In [14]:
adjacenyDict

{'66de0cab-0aca-11e7-82f6-3863bb44ef7c': ['66de4bbb-0aca-11e7-82f6-3863bb44ef7c',
  '66de4897-0aca-11e7-82f6-3863bb44ef7c',
  '66de5a47-0aca-11e7-82f6-3863bb44ef7c',
  '66de1295-0aca-11e7-82f6-3863bb44ef7c',
  '66de4959-0aca-11e7-82f6-3863bb44ef7c',
  '66de48f9-0aca-11e7-82f6-3863bb44ef7c',
  '66dcba5e-0aca-11e7-82f6-3863bb44ef7c',
  '66db88f5-0aca-11e7-82f6-3863bb44ef7c',
  '66de542a-0aca-11e7-82f6-3863bb44ef7c',
  '66de2b97-0aca-11e7-82f6-3863bb44ef7c'],
 '66dde76a-0aca-11e7-82f6-3863bb44ef7c': ['66dde630-0aca-11e7-82f6-3863bb44ef7c',
  '66ddef96-0aca-11e7-82f6-3863bb44ef7c',
  '66de0dfd-0aca-11e7-82f6-3863bb44ef7c',
  '66ddea2a-0aca-11e7-82f6-3863bb44ef7c',
  '66de13dc-0aca-11e7-82f6-3863bb44ef7c',
  '66dde845-0aca-11e7-82f6-3863bb44ef7c',
  '66de2a00-0aca-11e7-82f6-3863bb44ef7c',
  '66de677e-0aca-11e7-82f6-3863bb44ef7c',
  '66dd448f-0aca-11e7-82f6-3863bb44ef7c',
  '66dde208-0aca-11e7-82f6-3863bb44ef7c'],
 '43eac213-4603-4bc2-b6d6-b45013396282': ['999865d2-0735-4e85-a9d4-00810231530

In [15]:
#min max lat/long for normalize
# need to filter out zeros for some data noise 
minlat = min([i for i in d['lat'] if i > 0])
maxlat  = max([i for i in d['lat'] if i > 0])
minlong = min([i for i in d['lon'] if i  < 0])
maxlong = max([i for i in d['lon'] if i < 0])

#add two columns and normalize
#hardcoding column locations to overwrite ""
d['x'] = 0
d['y'] = 0
d['adjacency'] = ''
locData = d
locData = locData.reset_index()
# print(adjacenyDict)
for r in range(d.shape[0]):
    d.iloc[r,-3] = clamp((d.iloc[r, 1] - minlat)/(maxlat-minlat),0,1) #lat
    d.iloc[r,-2] = clamp((d.iloc[r, 2] - minlong)/(maxlong-minlong),0,1) #lat
    try: 
        d.iloc[r, -1] = ' '.join([str(locData.index[locData['station_id']==v][0]) for v in adjacenyDict[locData.iloc[r,0]]])
    except KeyError: pass

locData = d[['x', 'y', 'adjacency']]


In [16]:
locData.head()

,x,y,adjacency
station_id,,,
c71cca54-17f6-42bc-ba94-3f5bd9c70197,0.279596,0.497001,
038b7369-9c6e-4eb2-8569-f7461443302f,0.175610,0.905725,
66dce8db-0aca-11e7-82f6-3863bb44ef7c,0.298515,0.713951,916 1222 842 1031 1185 1095 1071 908 428 471
517409d3-a94a-4140-a6de-ff0de9221ed2,0.741003,0.884273,
66ddeaff-0aca-11e7-82f6-3863bb44ef7c,0.646029,0.548484,


In [7]:
locData.to_csv('../../clean/locdata.csv')

In [8]:
#get station indices
stationIndex1=0
stationIndex2=1200

#see https://ask.openrouteservice.org/t/point-not-within-a-radius-of-350-0-meters/1156/7
s1Routing = [d.iloc[stationIndex1][2], d.iloc[stationIndex1][1]] #lon, lat for ORS
s1Mapping = [d.iloc[stationIndex1][1], d.iloc[stationIndex1][2]] #lat, lon for folium

s2Routing = [d.iloc[stationIndex2][2], d.iloc[stationIndex2][1]] #lon, lat for ORS
s2Mapping = [d.iloc[stationIndex1][1], d.iloc[stationIndex1][2]] #lat, lon for folium

#setup our coords
coordinates = [s1Routing,s2Routing]

#hit api for route
route = client.directions(
    coordinates=coordinates,
    profile='cycling-road',
    format='geojson',
    options={"avoid_features": ["steps"]},
    validate=False,
)

#create list of loction points 
locations=[list(reversed(coord)) 
                           for coord in 
                           route['features'][0]['geometry']['coordinates']]

In [9]:
#min max lat/long for normalize
# need to filter out zeros for some data noise 
minlat = min([i for i in d['lat'] if i > 0])
maxlat  = max([i for i in d['lat'] if i > 0])
minlong = min([i for i in d['lon'] if i  < 0])
maxlong = max([i for i in d['lon'] if i < 0])

def normalizeToCitibikeRange(lat, lon):
    return clamp((lat - minlat)/(maxlat-minlat),0,1), clamp((lon-minlong)/(maxlong-minlong))

In [10]:
norm_locs = [[clamp((locations[i][0] - minlat)/(maxlat-minlat),0,1), clamp((locations[i][1]-minlong)/(maxlong-minlong),0,1)] for i in range(len(locations))]

In [11]:
norm_locs

[[0.2795941737820376, 0.4971173733978163],
 [0.2773962832747367, 0.4969517223005728],
 [0.27711099949774676, 0.49693223393617086],
 [0.2768819688598836, 0.4969176176629213],
 [0.2737800100451981, 0.49668862938138875],
 [0.27343043696634073, 0.4966642689259728],
 [0.27348267202411153, 0.49621603654514335],
 [0.2735951783023651, 0.4929566076022572],
 [0.2737920642892946, 0.4874316563000152],
 [0.2738081366147757, 0.4869980401925045],
 [0.27358312405826857, 0.4869931681014213],
 [0.2724460070316362, 0.4869736797370194],
 [0.2710999497739777, 0.4869054704617164],
 [0.27081868407837234, 0.4868908541883976],
 [0.27051330989454525, 0.48687623791514806],
 [0.2702280261175267, 0.48686649373291246],
 [0.27021597187343016, 0.4872416447472866],
 [0.27021597187343016, 0.487314726113742],
 [0.27020793571071816, 0.48752422603080336],
 [0.26995479658463334, 0.4876996213102825],
 [0.26879357106980784, 0.4885083884320981],
 [0.2685082872928179, 0.48872763253139506],
 [0.26850025113010584, 0.488498644249

In [12]:
#mapping with folium
m = folium.Map(location=s1Mapping, tiles='cartodbpositron', zoom_start=13)
folium.PolyLine(locations).add_to(m)
m

In [13]:
len(norm_locs)

708